In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

import matplotlib.pyplot as plt 
import numpy as np
import random
import json
import time

import math
from models import LogisticRegressor

In [2]:
def evaluate():        
    correct = 0
    total = 0
    # Iterate through test dataset
    for features, labels in test_loader:
        
        # Forward pass only to get logits/output
        outputs = model(features)
        
        # Get predictions from the maximum value
        _, predicted = torch.max(outputs.data, 1)
        
        # Total number of labels
        total += labels.size(0)
        
        # Total correct predictions
        correct += (predicted == labels).sum()
        
    accuracy = 100 * correct // total
    
    # Print Loss
    print("Loss: {}. Accuracy: {}".format(loss.item(), accuracy))


In [3]:
with open('data/mfccs.json') as f:
    json_data = json.load(f)

data = []
for video in json_data:
    true_count = 0
    false_count = 0
    for label in json_data[video]:
        if label == 'true':
            i_d = torch.tensor(1)
            for lis in json_data[video][label]:
                norm = np.linalg.norm(lis[:2])
                new = torch.FloatTensor(np.array(lis[:2]) / norm)
                result = (new, i_d)
                data.append(result)
                true_count += 1

        elif label == 'false':
            i_d = torch.tensor(0)
            for lis in json_data[video][label]:
                if false_count < true_count:
                    norm = np.linalg.norm(lis[:2])
                    new = torch.FloatTensor(np.array(lis[:2]) / norm)
                    result = (new, i_d)
                    data.append(result)
                    false_count +=1

print(data[:10])

[(tensor([0.8789, 0.4770]), tensor(1)), (tensor([-0.2110,  0.9775]), tensor(1)), (tensor([-0.1990,  0.9800]), tensor(1)), (tensor([0.6878, 0.7259]), tensor(1)), (tensor([0.3701, 0.9290]), tensor(1)), (tensor([0.5789, 0.8154]), tensor(1)), (tensor([-0.2460,  0.9693]), tensor(1)), (tensor([-0.3732,  0.9278]), tensor(1)), (tensor([-0.9657,  0.2595]), tensor(1)), (tensor([-0.1115,  0.9938]), tensor(1))]


In [4]:
train_set = data[:1700]
test_set = data[1700:]

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_set,
                                           batch_size=10,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set,
                                           batch_size=10,
                                           shuffle=True)

In [6]:
input_dim = 2
output_dim = 2
model = LogisticRegressor(input_dim, output_dim)

In [7]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [8]:
num_epochs = 10000
iters = 0
for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_loader):
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(features)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        iters += 1

        if iters % 1000 == 0:
            print(iters, end=' ')
            evaluate()


1000 Loss: 0.6530635952949524. Accuracy: 40
2000 Loss: 0.723237931728363. Accuracy: 42
3000 Loss: 0.7214637994766235. Accuracy: 42
4000 Loss: 0.743004322052002. Accuracy: 40
5000 Loss: 0.709425151348114. Accuracy: 44
6000 Loss: 0.7101415991783142. Accuracy: 40
7000 Loss: 0.6270326375961304. Accuracy: 40
8000 Loss: 0.7114494442939758. Accuracy: 43
9000 Loss: 0.6807051301002502. Accuracy: 53
10000 Loss: 0.7464209794998169. Accuracy: 50
11000 Loss: 0.7174743413925171. Accuracy: 47
12000 Loss: 0.7063252329826355. Accuracy: 41
13000 Loss: 0.5624274015426636. Accuracy: 49
14000 Loss: 0.6353782415390015. Accuracy: 42
15000 Loss: 0.59847491979599. Accuracy: 41
16000 Loss: 0.6182931661605835. Accuracy: 42
17000 Loss: 0.7379915714263916. Accuracy: 47
18000 Loss: 0.6925051212310791. Accuracy: 41
19000 Loss: 0.6243540048599243. Accuracy: 42
20000 Loss: 0.6592890024185181. Accuracy: 43
21000 Loss: 0.7000609636306763. Accuracy: 44
22000 Loss: 0.6705328822135925. Accuracy: 44
23000 Loss: 0.6703634262

KeyboardInterrupt: 